In [1]:
import numpy as np
import pandas as pd

In [2]:
# 按照 appendix 整理資料
# income 用 retirement income, 以 household 為單位乘以 1.48
# medical expenditure 要以 household total 計算

In [3]:
# 定義一些函數方便整理資料
def int_limit(ip, maximum=10*10, minimum=0):
    try:
        op = int(ip)
        if op > maximum or op < minimum:
            return None
        return op
    except:
        return None

def float_limit(ip, maximum=10*10, minimum=0):
    try:
        op = float(ip)
        if op > maximum or op < minimum:
            return None
        return op
    except:
        return None

In [4]:
# 1996, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\1996\h96da\H96A_R.DA', mode='r').read()))
B_R = np.array(list(open('D:\Desktop\imf\專題\data\\1996\h96da\H96B_R.DA', mode='r').read()))
CS_R = np.array(list(open('D:\Desktop\imf\專題\data\\1996\h96da\H96CS_R.DA', mode='r').read()))
R_R = np.array(list(open('D:\Desktop\imf\專題\data\\1996\h96da\H96R_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_1996', 'gender_1996', 'hl_sta_1996', 'mdc_expen_1996']
data_1996_R = pd.DataFrame(columns=cols)
for i in range(10964):
    age = int_limit(''.join(A_R[(147 * i + 91) : (147 * i + 94)]), 200)
    healthcare_AMT = float_limit(''.join(R_R[(355 * i + 42) : (355 * i + 46)]), 9996)
    healthcare_freq = { '1': 1, '2': 4, '3': 52, '4': 12 }.get(''.join(R_R[355 * i + 46]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(R_R[(355 * i + 257) : (355 * i + 266)]), 9994)
    LTC_freq = { '1': 1, '2': 4, '3': 52, '4': 12 }.get(''.join(R_R[355 * i + 266]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(147 * i) : (147 * i + 6)]),
        'pn': ''.join(A_R[(147 * i + 6) : (147 * i + 9)]),
        'age_1996': age,
        'gender_1996': { '1': 'Male', '2': 'Female' }.get(''.join(CS_R[392 * i + 45]), None),
        'hl_sta_1996': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(B_R[243 * i + 25]), None),
        'mdc_expen_1996': mdc_expen
    }, index=[0])
    data_1996_R = pd.concat([data_1996_R, new], ignore_index=True)

alter_data_1996_R = data_1996_R.dropna()
maps = alter_data_1996_R['mdc_expen_1996'].groupby(alter_data_1996_R['id']).mean()
alter_data_1996_R = alter_data_1996_R.drop(['mdc_expen_1996'], axis=1)
alter_data_1996_R.merge(maps, how='inner', on='id')

,id,pn,age_1996,gender_1996,hl_sta_1996,mdc_expen_1996
0,000003,010,60,Male,2,650.0
1,000003,020,58,Female,2,650.0
2,010004,010,57,Male,2,700.0
3,010004,040,50,Female,1,700.0
4,010013,040,49,Female,1,2628.0
...,...,...,...,...,...,...
813,090924,020,64,Female,1,816.0
814,092329,010,63,Female,3,1500.0
815,095033,010,56,Female,1,624.0
816,098097,030,68,Male,2,1865.0


Pension income：  
non-financial income, including (net of taxes)
* Social Security：321 - 327 (J_H)
* pension：635 - 640, 641 - 646 (J_H)
* other(disability)：1079 - 1085 (J_H)
* annuity：1310 - 1315 (J_H)
* government transfer income：  
    including
    * unemployment compensation：169 - 175 (J_H)
    * workers' compensation：245 - 251 (J_H)
    * supplemental security income：369 - 372 (J_H)
    * welfare：381 - 386 (J_H)
    * veteran benefits or military pension：389 - 394 (J_H)
* labor income：48 - 54, 60 - 66, 72 - 78, 84 - 90, 96 - 102 (J_H)

In [5]:
# 1996, 以 household 為單位的回答
J_H = np.array(list(open('D:\Desktop\imf\專題\data\\1996\h96da\H96J_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_1996']
data_1996_H = pd.DataFrame(columns=cols)
for i in range(6816):
    ss = int_limit(''.join(J_H[(1350 * i + 320) : (1350 * i + 327)]), 9999996)
    p1 = int_limit(''.join(J_H[(1350 * i + 560) : (1350 * i + 567)]), 999996)
    p2 = int_limit(''.join(J_H[(1350 * i + 567) : (1350 * i + 574)]), 999996)
    o = int_limit(''.join(J_H[(1350 * i + 1078) : (1350 * i + 1085)]), 999996)
    ann = int_limit(''.join(J_H[(1350 * i + 1309) : (1350 * i + 1315)]), 999996)
    un_c = int_limit(''.join(J_H[(1350 * i + 168) : (1350 * i + 175)]), 9999996, 1)
    w_c = int_limit(''.join(J_H[(1350 * i + 244) : (1350 * i + 251)]), 9999996, 1)
    ssi = int_limit(''.join(J_H[(1350 * i + 368) : (1350 * i + 372)]), 9996)
    wel = int_limit(''.join(J_H[(1350 * i + 380) : (1350 * i + 386)]), 999996, 1)
    vbmp = int_limit(''.join(J_H[(1350 * i + 388) : (1350 * i + 394)]), 999996)
    li1 = int_limit(''.join(J_H[(1350 * i + 47) : (1350 * i + 54)]), 9999996)
    li2 = int_limit(''.join(J_H[(1350 * i + 59) : (1350 * i + 66)]), 9999996)
    li3 = int_limit(''.join(J_H[(1350 * i + 71) : (1350 * i + 78)]), 9999996)
    li4 = int_limit(''.join(J_H[(1350 * i + 83) : (1350 * i + 90)]), 9999996)
    li5 = int_limit(''.join(J_H[(1350 * i + 95) : (1350 * i + 102)]), 9999996)
    filter_list = filter(None, [ss, p1, p2, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(J_H[(1350 * i) : (1350 * i + 6)]),
        'subhh': ''.join(J_H[1350 * i + 6]),
        'income_1996': income
    }, index=[0])
    data_1996_H = pd.concat([data_1996_H, new], ignore_index=True)
    
alter_data_1996_H = data_1996_H.dropna()
maps = alter_data_1996_H['income_1996'].groupby(alter_data_1996_H['id']).mean()
print(maps)

id
000002       483.0
000003      4000.0
010001      4224.0
010003         0.0
010004    261205.0
            ...   
098121       400.0
098802       844.0
098859       575.0
099231     40000.0
099564     40000.0
Name: income_1996, Length: 6696, dtype: float64


In [6]:
alter_data_1996 = alter_data_1996_R.merge(maps, how='inner', on='id')
alter_data_1996['income_1996'] = alter_data_1996.apply(lambda x: x['income_1996'] if x['hl_sta_1996'] == 1 else x['income_1996'] * 1.48, axis=1)
print(alter_data_1996)

         id   pn age_1996 gender_1996 hl_sta_1996  income_1996
0    000003  010       60        Male           2      5920.00
1    000003  020       58      Female           2      5920.00
2    010004  010       57        Male           2    386583.40
3    010004  040       50      Female           1    261205.00
4    010013  040       49      Female           1    539996.50
..      ...  ...      ...         ...         ...          ...
813  090924  020       64      Female           1      8125.00
814  092329  010       63      Female           3     89522.24
815  095033  010       56      Female           1     27005.00
816  098097  030       68        Male           2     32715.40
817  099564  010       62      Female           1     40000.00

[818 rows x 6 columns]


In [7]:
# 1998, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\1998\h98da\H98A_R.DA', mode='r').read()))
B_R = np.array(list(open('D:\Desktop\imf\專題\data\\1998\h98da\H98B_R.DA', mode='r').read()))
CS_R = np.array(list(open('D:\Desktop\imf\專題\data\\1998\h98da\H98CS_R.DA', mode='r').read()))
R_R = np.array(list(open('D:\Desktop\imf\專題\data\\1998\h98da\H98R_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_1998', 'gender_1998', 'hl_sta_1998', 'mdc_expen_1998']
data_1998_R = pd.DataFrame(columns=cols)
for i in range(21384):
    age = int_limit(''.join(A_R[(193 * i + 98) : (193 * i + 101)]), 200)
    healthcare_AMT = float_limit(''.join(R_R[(312 * i + 60) : (312 * i + 64)]), 9996)
    healthcare_freq = { '1': 1, '2': 4, '3': 6, '4': 12, '5': 52, '6': 26, '7': 2, '8': 24 }.get(''.join(R_R[(312 * i + 64) : (312 * i + 66)]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(R_R[(312 * i + 214) : (312 * i + 223)]), 9996)
    LTC_freq = { '1': 1, '2': 4, '3': 52, '4': 12 }.get(''.join(R_R[312 * i + 223]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(193 * i) : (193 * i + 6)]),
        'pn': ''.join(A_R[(193 * i + 6) : (193 * i + 9)]),
        'age_1998': age,
        'gender_1998': { '1': 'Male', '2': 'Female' }.get(''.join(CS_R[153 * i + 37]), None),
        'hl_sta_1998': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(B_R[255 * i + 24]), None),
        'mdc_expen_1998': mdc_expen
    }, index=[0])
    data_1998_R = pd.concat([data_1998_R, new], ignore_index=True)

alter_data_1998_R = data_1998_R.dropna()
maps = alter_data_1998_R['mdc_expen_1998'].groupby(alter_data_1998_R['id']).mean()
alter_data_1998_R = alter_data_1998_R.drop(['mdc_expen_1998'], axis=1)
alter_data_1998_R.merge(maps, how='inner', on='id')

# 1998, 以 household 為單位的回答
J_H = np.array(list(open('D:\Desktop\imf\專題\data\\1998\h98da\H98J_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_1998']
data_1998_H = pd.DataFrame(columns=cols)
for i in range(14395):
    ss = int_limit(''.join(J_H[(1316 * i + 215) : (1316 * i + 221)]), 999996)
    p1 = int_limit(''.join(J_H[(1316 * i + 560) : (1316 * i + 567)]), 9999996)
    p2 = int_limit(''.join(J_H[(1316 * i + 567) : (1316 * i + 574)]), 9999996)
    o = int_limit(''.join(J_H[(1316 * i + 1056) : (1316 * i + 1063)]), 9999996)
    ann = int_limit(''.join(J_H[(1316 * i + 510) : (1316 * i + 518)]), 99999996)
    un_c = int_limit(''.join(J_H[(1316 * i + 183) : (1316 * i + 190)]), 9999996)
    w_c = int_limit(''.join(J_H[(1316 * i + 199) : (1316 * i + 206)]), 9999996)
    ssi = int_limit(''.join(J_H[(1316 * i + 267) : (1316 * i + 271)]), 9996)
    wel = int_limit(''.join(J_H[(1316 * i + 282) : (1316 * i + 288)]), 999996)
    vbmp = int_limit(''.join(J_H[(1316 * i + 388) : (1316 * i + 394)]), 999996)
    li1 = int_limit(''.join(J_H[(1316 * i + 48) : (1316 * i + 56)]), 99999996)
    li2 = int_limit(''.join(J_H[(1316 * i + 63) : (1316 * i + 71)]), 99999996)
    li3 = int_limit(''.join(J_H[(1316 * i + 78) : (1316 * i + 86)]), 99999996)
    li4 = int_limit(''.join(J_H[(1316 * i + 91) : (1316 * i + 99)]), 99999996)
    li5 = int_limit(''.join(J_H[(1316 * i + 104) : (1316 * i + 112)]), 99999996)
    filter_list = filter(None, [ss, p1, p2, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(J_H[(1316 * i) : (1316 * i + 6)]),
        'subhh': ''.join(J_H[1316 * i + 6]),
        'income_1998': income
    }, index=[0])
    data_1998_H = pd.concat([data_1998_H, new], ignore_index=True)
    
alter_data_1998_H = data_1998_H.dropna()
maps = alter_data_1998_H['income_1998'].groupby(alter_data_1998_H['id']).mean()

alter_data_1998 = alter_data_1998_R.merge(maps, how='inner', on='id')
alter_data_1998['income_1998'] = alter_data_1998.apply(lambda x: x['income_1998'] if x['hl_sta_1998'] == 1 else x['income_1998'] * 1.48, axis=1)
print(alter_data_1998)

          id   pn age_1998 gender_1998 hl_sta_1998  income_1998
0     000003  010       62        Male           2      7518.40
1     000003  020       60      Female           2      7518.40
2     010004  010       59        Male           2      5994.00
3     010004  040       52      Female           1      4050.00
4     010050  010       57      Female           1     71800.00
...      ...  ...      ...         ...         ...          ...
5376  213410  010       69        Male           1    200400.00
5377  213435  010       74      Female           2      1092.24
5378  213467  020       70        Male           2     11939.16
5379  213475  010       73      Female           2      1626.52
5380  213479  010       71        Male           3     14931.72

[5381 rows x 6 columns]


In [8]:
# 2000, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2000\h00da\H00A_R.DA', mode='r').read()))
B_R = np.array(list(open('D:\Desktop\imf\專題\data\\2000\h00da\H00B_R.DA', mode='r').read()))
CS_R = np.array(list(open('D:\Desktop\imf\專題\data\\2000\h00da\H00CS_R.DA', mode='r').read()))
R_R = np.array(list(open('D:\Desktop\imf\專題\data\\2000\h00da\H00R_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2000', 'gender_2000', 'hl_sta_2000', 'mdc_expen_2000']
data_2000_R = pd.DataFrame(columns=cols)
for i in range(19580):
    age = int_limit(''.join(A_R[(195 * i + 99) : (195 * i + 102)]), 200)
    healthcare_AMT = float_limit(''.join(R_R[(379 * i + 64) : (379 * i + 69)]), 9996)
    healthcare_freq = { '1': 1, '2': 4, '3': 6, '4': 12, '5': 52, '6': 26, '7': 2, '8': 24 }.get(''.join(R_R[(379 * i + 69) : (379 * i + 71)]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(R_R[(379 * i + 229) : (379 * i + 235)]), 9996)
    LTC_freq = { '1': 1, '2': 4, '3': 52, '4': 12 }.get(''.join(R_R[379 * i + 235]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(195 * i) : (195 * i + 6)]),
        'pn': ''.join(A_R[(195 * i + 6) : (195 * i + 9)]),
        'age_2000': age,
        'gender_2000': { '1': 'Male', '2': 'Female' }.get(''.join(CS_R[168 * i + 42]), None),
        'hl_sta_2000': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(B_R[251 * i + 24]), None),
        'mdc_expen_2000': mdc_expen
    }, index=[0])
    data_2000_R = pd.concat([data_2000_R, new], ignore_index=True)

alter_data_2000_R = data_2000_R.dropna()
maps = alter_data_2000_R['mdc_expen_2000'].groupby(alter_data_2000_R['id']).mean()
alter_data_2000_R = alter_data_2000_R.drop(['mdc_expen_2000'], axis=1)
alter_data_2000_R.merge(maps, how='inner', on='id')

# 2000, 以 household 為單位的回答
J_H = np.array(list(open('D:\Desktop\imf\專題\data\\2000\h00da\H00J_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2000']
data_2000_H = pd.DataFrame(columns=cols)
for i in range(13214):
    ss = int_limit(''.join(J_H[(1298 * i + 215) : (1298 * i + 219)]), 9996)
    p1 = int_limit(''.join(J_H[(1298 * i + 556) : (1298 * i + 563)]), 9999996)
    p2 = int_limit(''.join(J_H[(1298 * i + 563) : (1298 * i + 570)]), 9999996)
    o = int_limit(''.join(J_H[(1298 * i + 624) : (1298 * i + 631)]), 9999996)
    ann = int_limit(''.join(J_H[(1298 * i + 502) : (1298 * i + 510)]), 99999996)
    un_c = int_limit(''.join(J_H[(1298 * i + 183) : (1298 * i + 190)]), 9999996)
    w_c = int_limit(''.join(J_H[(1298 * i + 199) : (1298 * i + 206)]), 9999996)
    ssi = int_limit(''.join(J_H[(1298 * i + 263) : (1298 * i + 267)]), 9996)
    wel = int_limit(''.join(J_H[(1298 * i + 278) : (1298 * i + 284)]), 999996)
    vbmp = int_limit(''.join(J_H[(1298 * i + 286) : (1298 * i + 290)]), 9996)
    li1 = int_limit(''.join(J_H[(1298 * i + 48) : (1298 * i + 56)]), 99999996)
    li2 = int_limit(''.join(J_H[(1298 * i + 63) : (1298 * i + 71)]), 99999996)
    li3 = int_limit(''.join(J_H[(1298 * i + 78) : (1298 * i + 86)]), 99999996)
    li4 = int_limit(''.join(J_H[(1298 * i + 91) : (1298 * i + 99)]), 99999996)
    li5 = int_limit(''.join(J_H[(1298 * i + 104) : (1298 * i + 112)]), 99999996)
    filter_list = filter(None, [ss, p1, p2, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(J_H[(1298 * i) : (1298 * i + 6)]),
        'subhh': ''.join(J_H[1298 * i + 6]),
        'income_2000': income
    }, index=[0])
    data_2000_H = pd.concat([data_2000_H, new], ignore_index=True)
    
alter_data_2000_H = data_2000_H.dropna()
maps = alter_data_2000_H['income_2000'].groupby(alter_data_2000_H['id']).mean()

alter_data_2000 = alter_data_2000_R.merge(maps, how='inner', on='id')
alter_data_2000['income_2000'] = alter_data_2000.apply(lambda x: x['income_2000'] if x['hl_sta_2000'] == 1 else x['income_2000'] * 1.48, axis=1)
print(alter_data_2000)

          id   pn age_2000 gender_2000 hl_sta_2000  income_2000
0     000003  010       64        Male           2       6216.0
1     000003  020       61      Female           2       6216.0
2     010004  010       60        Male           2       7400.0
3     010004  040       54      Female           1       5000.0
4     010038  010       63        Male           1     170000.0
...      ...  ...      ...         ...         ...          ...
4914  213435  010       75      Female           2       1036.0
4915  213442  020       72        Male           1        696.0
4916  213467  010       71      Female           1       7700.0
4917  213467  020       71        Male           2      11396.0
4918  213479  010       73        Male           2       1480.0

[4919 rows x 6 columns]


In [9]:
# 2002, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2002\h02da\H02A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2002\h02da\H02C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2002\h02da\H02PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2002\h02da\H02N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2002', 'gender_2002', 'hl_sta_2002', 'mdc_expen_2002']
data_2002_R = pd.DataFrame(columns=cols)
for i in range(18167):
    age = int_limit(''.join(A_R[(67 * i + 30) : (67 * i + 33)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(644 * i + 34) : (644 * i + 38)]), 9996)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(''.join(N_R[644 * i + 38]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(644 * i + 240) : (644 * i + 246)]), 9996)
    LTC_freq = { '1': 1, '2': 4, '4': 12, '6': 1 }.get(''.join(N_R[644 * i + 246]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(67 * i) : (67 * i + 6)]),
        'pn': ''.join(A_R[(67 * i + 6) : (67 * i + 9)]),
        'age_2002': age,
        'gender_2002': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[414 * i + 19]), None),
        'hl_sta_2002': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[256 * i + 17]), None),
        'mdc_expen_2002': mdc_expen
    }, index=[0])
    data_2002_R = pd.concat([data_2002_R, new], ignore_index=True)

alter_data_2002_R = data_2002_R.dropna()
maps = alter_data_2002_R['mdc_expen_2002'].groupby(alter_data_2002_R['id']).mean()
alter_data_2002_R = alter_data_2002_R.drop(['mdc_expen_2002'], axis=1)
alter_data_2002_R.merge(maps, how='inner', on='id')

# 2002, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2002\h02da\H02Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2002']
data_2002_H = pd.DataFrame(columns=cols)
for i in range(12350):
    ss = int_limit(''.join(Q_H[(2245 * i + 309) : (2245 * i + 313)]), 9996)
    p = int_limit(''.join(Q_H[(2245 * i + 815) : (2245 * i + 820)]), 99996)
    o = int_limit(''.join(Q_H[(2245 * i + 867) : (2245 * i + 871)]), 9996)
    ann = int_limit(''.join(Q_H[(2245 * i + 1059) : (2245 * i + 1064)]), 99996)
    un_c = int_limit(''.join(Q_H[(2245 * i + 241) : (2245 * i + 246)]), 99996)
    w_c = int_limit(''.join(Q_H[(2245 * i + 275) : (2245 * i + 280)]), 99996)
    ssi = int_limit(''.join(Q_H[(2245 * i + 354) : (2245 * i + 358)]), 99996)
    wel = int_limit(''.join(Q_H[(2245 * i + 375) : (2245 * i + 380)]), 99996)
    vbmp = int_limit(''.join(Q_H[(2245 * i + 393) : (2245 * i + 397)]), 9996)
    li1 = int_limit(''.join(Q_H[(2245 * i + 54) : (2245 * i + 60)]), 999996)
    li2 = int_limit(''.join(Q_H[(2245 * i + 76) : (2245 * i + 82)]), 999996)
    li3 = int_limit(''.join(Q_H[(2245 * i + 98) : (2245 * i + 104)]), 999996)
    li4 = int_limit(''.join(Q_H[(2245 * i + 118) : (2245 * i + 123)]), 999996)
    li5 = int_limit(''.join(Q_H[(2245 * i + 137) : (2245 * i + 143)]), 999996)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(2245 * i) : (2245 * i + 6)]),
        'subhh': ''.join(Q_H[2245 * i + 6]),
        'income_2002': income
    }, index=[0])
    data_2002_H = pd.concat([data_2002_H, new], ignore_index=True)
    
alter_data_2002_H = data_2002_H.dropna()
maps = alter_data_2002_H['income_2002'].groupby(alter_data_2002_H['id']).mean()

alter_data_2002 = alter_data_2002_R.merge(maps, how='inner', on='id')
alter_data_2002['income_2002'] = alter_data_2002.apply(lambda x: x['income_2002'] if x['hl_sta_2002'] == 1 else x['income_2002'] * 1.48, axis=1)
print(alter_data_2002)

          id   pn age_2002 gender_2002 hl_sta_2002  income_2002
0     000003  010       66        Male           2     21821.12
1     000003  020       63      Female           2     21821.12
2     010004  010       62        Male           1      4300.00
3     010004  040       56      Female           1      4300.00
4     010038  010       65        Male           1      1522.00
...      ...  ...      ...         ...         ...          ...
2995  213443  010       74        Male           3     15688.00
2996  213446  010       75      Female           3       740.00
2997  213446  020       75      Female           1       500.00
2998  213449  010       72        Male           2       510.60
2999  213465  010       78      Female           1       700.00

[3000 rows x 6 columns]


In [10]:
# 2004, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2004\h04da\H04A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2004\h04da\H04C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2004\h04da\H04PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2004\h04da\H04N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2004', 'gender_2004', 'hl_sta_2004', 'mdc_expen_2004']
data_2004_R = pd.DataFrame(columns=cols)
for i in range(20129):
    age = int_limit(''.join(A_R[(57 * i + 42) : (57 * i + 45)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(674 * i + 33) : (674 * i + 37)]), 1000)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(''.join(N_R[644 * i + 37]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(674 * i + 248) : (674 * i + 252)]), 9996)
    LTC_freq = { '1': 1, '2': 4, '4': 12, '6': 1 }.get(''.join(N_R[644 * i + 252]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(57 * i) : (57 * i + 6)]),
        'pn': ''.join(A_R[(57 * i + 6) : (57 * i + 9)]),
        'age_2004': age,
        'gender_2004': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[546 * i + 20]), None),
        'hl_sta_2004': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[276 * i + 17]), None),
        'mdc_expen_2004': mdc_expen
    }, index=[0])
    data_2004_R = pd.concat([data_2004_R, new], ignore_index=True)

alter_data_2004_R = data_2004_R.dropna()
maps = alter_data_2004_R['mdc_expen_2004'].groupby(alter_data_2004_R['id']).mean()
alter_data_2004_R = alter_data_2004_R.drop(['mdc_expen_2004'], axis=1)
alter_data_2004_R.merge(maps, how='inner', on='id')

# 2004, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2004\h04da\H04Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2004']
data_2004_H = pd.DataFrame(columns=cols)
for i in range(13645):
    ss = int_limit(''.join(Q_H[(2336 * i + 334) : (2336 * i + 338)]), 9998)
    p = int_limit(''.join(Q_H[(2336 * i + 868) : (2336 * i + 874)]), 999998)
    o = int_limit(''.join(Q_H[(2336 * i + 983) : (2336 * i + 987)]), 9998)
    ann = int_limit(''.join(Q_H[(2336 * i + 1160) : (2336 * i + 1166)]), 999998)
    un_c = int_limit(''.join(Q_H[(2336 * i + 257) : (2336 * i + 262)]), 99998)
    w_c = int_limit(''.join(Q_H[(2336 * i + 297) : (2336 * i + 302)]), 99998)
    ssi = int_limit(''.join(Q_H[(2336 * i + 385) : (2336 * i + 389)]), 9998)
    wel = int_limit(''.join(Q_H[(2336 * i + 405) : (2336 * i + 410)]), 99998)
    vbmp = int_limit(''.join(Q_H[(2336 * i + 426) : (2336 * i + 430)]), 9998)
    li1 = int_limit(''.join(Q_H[(2336 * i + 31) : (2336 * i + 38)]), 9999998)
    li2 = int_limit(''.join(Q_H[(2336 * i + 55) : (2336 * i + 62)]), 9999998)
    li3 = int_limit(''.join(Q_H[(2336 * i + 79) : (2336 * i + 85)]), 999998)
    li4 = int_limit(''.join(Q_H[(2336 * i + 102) : (2336 * i + 108)]), 999998)
    li5 = int_limit(''.join(Q_H[(2336 * i + 124) : (2336 * i + 129)]), 99998)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(2336 * i) : (2336 * i + 6)]),
        'subhh': ''.join(Q_H[2336 * i + 6]),
        'income_2004': income
    }, index=[0])
    data_2004_H = pd.concat([data_2004_H, new], ignore_index=True)
    
alter_data_2004_H = data_2004_H.dropna()
maps = alter_data_2004_H['income_2004'].groupby(alter_data_2004_H['id']).mean()

alter_data_2004 = alter_data_2004_R.merge(maps, how='inner', on='id')
alter_data_2004['income_2004'] = alter_data_2004.apply(lambda x: x['income_2004'] if x['hl_sta_2004'] == 1 else x['income_2004'] * 1.48, axis=1)
print(alter_data_2004)

          id   pn age_2004 gender_2004 hl_sta_2004  income_2004
0     000003  010       68        Male           2     13172.00
1     000003  020       65      Female           2     13172.00
2     010004  040       58      Female           1      5450.00
3     010038  010       67        Male           1    143600.00
4     010075  030       76        Male           2       435.12
...      ...  ...      ...         ...         ...          ...
2369  502635  020       50      Female           1     75000.00
2370  502657  010       54        Male           1         0.00
2371  502657  020       52      Female           2         0.00
2372  502660  020       46        Male           2    384800.00
2373  502711  020       43      Female           1     36000.00

[2374 rows x 6 columns]


In [11]:
# 2006, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2006\h06da\H06A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2006\h06da\H06C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2006\h06da\H06PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2006\h06da\H06N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2006', 'gender_2006', 'hl_sta_2006', 'mdc_expen_2006']
data_2006_R = pd.DataFrame(columns=cols)
for i in range(18469):
    age = int_limit(''.join(A_R[(139 * i + 54) : (139 * i + 57)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(980 * i + 154) : (980 * i + 157)]), 938)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(''.join(N_R[980 * i + 157]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(980 * i + 398) : (980 * i + 404)]), 158915)
    LTC_freq = { '1': 12, '2': 4, '3': 52, '4': 1, '6': 1 }.get(''.join(N_R[980 * i + 404]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(139 * i) : (139 * i + 6)]),
        'pn': ''.join(A_R[(139 * i + 6) : (139 * i + 9)]),
        'age_2006': age,
        'gender_2006': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[659 * i + 20]), None),
        'hl_sta_2006': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[325 * i + 17]), None),
        'mdc_expen_2006': mdc_expen
    }, index=[0])
    data_2006_R = pd.concat([data_2006_R, new], ignore_index=True)

alter_data_2006_R = data_2006_R.dropna()
maps = alter_data_2006_R['mdc_expen_2006'].groupby(alter_data_2006_R['id']).mean()
alter_data_2006_R = alter_data_2006_R.drop(['mdc_expen_2006'], axis=1)
alter_data_2006_R.merge(maps, how='inner', on='id')

# 2006, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2006\h06da\H06Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2006']
data_2006_H = pd.DataFrame(columns=cols)
for i in range(12605):
    ss = int_limit(''.join(Q_H[(3109 * i + 432) : (3109 * i + 436)]), 9998)
    p = int_limit(''.join(Q_H[(3109 * i + 1152) : (3109 * i + 1159)]), 9999998)
    o = int_limit(''.join(Q_H[(3109 * i + 1297) : (3109 * i + 1304)]), 9999998)
    ann = int_limit(''.join(Q_H[(3109 * i + 1550) : (3109 * i + 1557)]), 9999998)
    un_c = int_limit(''.join(Q_H[(3109 * i + 322) : (3109 * i + 330)]), 99999998)
    w_c = int_limit(''.join(Q_H[(3109 * i + 380) : (3109 * i + 387)]), 9999998)
    ssi = int_limit(''.join(Q_H[(3109 * i + 487) : (3109 * i + 491)]), 9998)
    wel = int_limit(''.join(Q_H[(3109 * i + 515) : (3109 * i + 521)]), 999998)
    vbmp = int_limit(''.join(Q_H[(3109 * i + 541) : (3109 * i + 545)]), 9998)
    li1 = int_limit(''.join(Q_H[(3109 * i + 31) : (3109 * i + 39)]), 99999998)
    li2 = int_limit(''.join(Q_H[(3109 * i + 60) : (3109 * i + 68)]), 99999998)
    li3 = int_limit(''.join(Q_H[(3109 * i + 89) : (3109 * i + 97)]), 99999998)
    li4 = int_limit(''.join(Q_H[(3109 * i + 118) : (3109 * i + 126)]), 99999998)
    li5 = int_limit(''.join(Q_H[(3109 * i + 147) : (3109 * i + 155)]), 99999998)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(3109 * i) : (3109 * i + 6)]),
        'subhh': ''.join(Q_H[3109 * i + 6]),
        'income_2006': income
    }, index=[0])
    data_2006_H = pd.concat([data_2006_H, new], ignore_index=True)
    
alter_data_2006_H = data_2006_H.dropna()
maps = alter_data_2006_H['income_2006'].groupby(alter_data_2006_H['id']).mean()

alter_data_2006 = alter_data_2006_R.merge(maps, how='inner', on='id')
alter_data_2006['income_2006'] = alter_data_2006.apply(lambda x: x['income_2006'] if x['hl_sta_2006'] == 1 else x['income_2006'] * 1.48, axis=1)
print(alter_data_2006)

          id   pn age_2006 gender_2006 hl_sta_2006  income_2006
0     000003  010       70        Male           2       4099.6
1     000003  020       67      Female           2       4099.6
2     010004  010       66        Male           2       7252.0
3     010004  040       60      Female           1       4900.0
4     010038  010       70        Male           1     101700.0
...      ...  ...      ...         ...         ...          ...
2309  502635  010       52        Male           1      88000.0
2310  502635  020       53      Female           1      88000.0
2311  502657  010       56        Male           1          0.0
2312  502657  020       54      Female           2          0.0
2313  502723  010       57        Male           2      42180.0

[2314 rows x 6 columns]


In [12]:
# 2008, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2008\h08da\H08A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2008\h08da\H08C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2008\h08da\H08PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2008\h08da\H08N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2008', 'gender_2008', 'hl_sta_2008', 'mdc_expen_2008']
data_2008_R = pd.DataFrame(columns=cols)
for i in range(17217):
    age = int_limit(''.join(A_R[(131 * i + 54) : (131 * i + 57)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(860 * i + 34) : (860 * i + 38)]), 4600)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(''.join(N_R[860 * i + 52]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(860 * i + 354) : (860 * i + 360)]), 100000)
    LTC_freq = { '1': 12, '2': 4, '3': 52, '4': 1, '6': 1 }.get(''.join(N_R[860 * i + 378]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(131 * i) : (131 * i + 6)]),
        'pn': ''.join(A_R[(131 * i + 6) : (131 * i + 9)]),
        'age_2008': age,
        'gender_2008': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[674 * i + 20]), None),
        'hl_sta_2008': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[408 * i + 68]), None),
        'mdc_expen_2008': mdc_expen
    }, index=[0])
    data_2008_R = pd.concat([data_2008_R, new], ignore_index=True)

alter_data_2008_R = data_2008_R.dropna()
maps = alter_data_2008_R['mdc_expen_2008'].groupby(alter_data_2008_R['id']).mean()
alter_data_2008_R = alter_data_2008_R.drop(['mdc_expen_2008'], axis=1)
alter_data_2008_R.merge(maps, how='inner', on='id')

# 2008, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2008\h08da\H08Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2008']
data_2008_H = pd.DataFrame(columns=cols)
for i in range(11897):
    ss = int_limit(''.join(Q_H[(3131 * i + 432) : (3131 * i + 436)]), 9998)
    p = int_limit(''.join(Q_H[(3131 * i + 1130) : (3131 * i + 1137)]), 9999998)
    o = int_limit(''.join(Q_H[(3131 * i + 1275) : (3131 * i + 1282)]), 9999998)
    ann = int_limit(''.join(Q_H[(3131 * i + 1528) : (3131 * i + 1535)]), 9999998)
    un_c = int_limit(''.join(Q_H[(3131 * i + 322) : (3131 * i + 330)]), 99999998)
    w_c = int_limit(''.join(Q_H[(3131 * i + 380) : (3131 * i + 387)]), 9999998)
    ssi = int_limit(''.join(Q_H[(3131 * i + 487) : (3131 * i + 491)]), 9998)
    wel = int_limit(''.join(Q_H[(3131 * i + 515) : (3131 * i + 521)]), 999998)
    vbmp = int_limit(''.join(Q_H[(3131 * i + 541) : (3131 * i + 545)]), 9998)
    li1 = int_limit(''.join(Q_H[(3131 * i + 31) : (3131 * i + 39)]), 99999998)
    li2 = int_limit(''.join(Q_H[(3131 * i + 60) : (3131 * i + 68)]), 99999998)
    li3 = int_limit(''.join(Q_H[(3131 * i + 89) : (3131 * i + 97)]), 99999998)
    li4 = int_limit(''.join(Q_H[(3131 * i + 118) : (3131 * i + 126)]), 99999998)
    li5 = int_limit(''.join(Q_H[(3131 * i + 147) : (3131 * i + 155)]), 99999998)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(3131 * i) : (3131 * i + 6)]),
        'subhh': ''.join(Q_H[3131 * i + 6]),
        'income_2008': income
    }, index=[0])
    data_2008_H = pd.concat([data_2008_H, new], ignore_index=True)
    
alter_data_2008_H = data_2008_H.dropna()
maps = alter_data_2008_H['income_2008'].groupby(alter_data_2008_H['id']).mean()

alter_data_2008 = alter_data_2008_R.merge(maps, how='inner', on='id')
alter_data_2008['income_2008'] = alter_data_2008.apply(lambda x: x['income_2008'] if x['hl_sta_2008'] == 1 else x['income_2008'] * 1.48, axis=1)
print(alter_data_2008)

          id   pn age_2008 gender_2008 hl_sta_2008  income_2008
0     000003  010       72        Male           2       3848.0
1     000003  020       69      Female           2       3848.0
2     010004  010       68        Male           2       7030.0
3     010004  040       61      Female           1       4750.0
4     010038  010       71        Male           1      61859.0
...      ...  ...      ...         ...         ...          ...
2417  502635  020       54      Female           1     110000.0
2418  502657  010       57        Male           1       6000.0
2419  502657  020       56      Female           1       6000.0
2420  502747  010       55      Female           2      36852.0
2421  502747  020       54        Male           2      36852.0

[2422 rows x 6 columns]


In [13]:
# 2010, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2010\h10da\H10A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2010\h10da\H10C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2010\h10da\H10PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2010\h10da\H10N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2010', 'gender_2010', 'hl_sta_2010', 'mdc_expen_2010']
data_2010_R = pd.DataFrame(columns=cols)
for i in range(22034):
    age = int_limit(''.join(A_R[(66 * i + 42) : (66 * i + 45)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(774 * i + 38) : (774 * i + 42)]), 1000)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(''.join(N_R[774 * i + 55]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(774 * i + 321) : (774 * i + 327)]), 100000)
    LTC_freq = { '1': 12, '2': 4, '3': 52, '4': 1, '6': 1 }.get(''.join(N_R[774 * i + 340]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(66 * i) : (66 * i + 6)]),
        'pn': ''.join(A_R[(66 * i + 6) : (66 * i + 9)]),
        'age_2010': age,
        'gender_2010': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[500 * i + 20]), None),
        'hl_sta_2010': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[365 * i + 23]), None),
        'mdc_expen_2010': mdc_expen
    }, index=[0])
    data_2010_R = pd.concat([data_2010_R, new], ignore_index=True)

alter_data_2010_R = data_2010_R.dropna()
maps = alter_data_2010_R['mdc_expen_2010'].groupby(alter_data_2010_R['id']).mean()
alter_data_2010_R = alter_data_2010_R.drop(['mdc_expen_2010'], axis=1)
alter_data_2010_R.merge(maps, how='inner', on='id')

# 2010, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2010\h10da\H10Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2010']
data_2010_H = pd.DataFrame(columns=cols)
for i in range(15280):
    ss = int_limit(''.join(Q_H[(2538 * i + 367) : (2538 * i + 372)]), 99998)
    p = int_limit(''.join(Q_H[(2538 * i + 944) : (2538 * i + 951)]), 9999998)
    o = int_limit(''.join(Q_H[(2538 * i + 1065) : (2538 * i + 1072)]), 9999998)
    ann = int_limit(''.join(Q_H[(2538 * i + 1255) : (2538 * i + 1262)]), 9999998)
    un_c = int_limit(''.join(Q_H[(2538 * i + 277) : (2538 * i + 285)]), 99999998)
    w_c = int_limit(''.join(Q_H[(2538 * i + 323) : (2538 * i + 330)]), 9999998)
    ssi = int_limit(''.join(Q_H[(2538 * i + 419) : (2538 * i + 423)]), 9998)
    wel = int_limit(''.join(Q_H[(2538 * i + 444) : (2538 * i + 450)]), 999998)
    vbmp = int_limit(''.join(Q_H[(2538 * i + 466) : (2538 * i + 470)]), 9998)
    li1 = int_limit(''.join(Q_H[(2538 * i + 31) : (2538 * i + 39)]), 99999998)
    li2 = int_limit(''.join(Q_H[(2538 * i + 56) : (2538 * i + 64)]), 99999998)
    li3 = int_limit(''.join(Q_H[(2538 * i + 81) : (2538 * i + 89)]), 99999998)
    li4 = int_limit(''.join(Q_H[(2538 * i + 106) : (2538 * i + 114)]), 99999998)
    li5 = int_limit(''.join(Q_H[(2538 * i + 130) : (2538 * i + 138)]), 99999998)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(2538 * i) : (2538 * i + 6)]),
        'subhh': ''.join(Q_H[2538 * i + 6]),
        'income_2010': income
    }, index=[0])
    data_2010_H = pd.concat([data_2010_H, new], ignore_index=True)
    
alter_data_2010_H = data_2010_H.dropna()
maps = alter_data_2010_H['income_2010'].groupby(alter_data_2010_H['id']).mean()

alter_data_2010 = alter_data_2010_R.merge(maps, how='inner', on='id')
alter_data_2010['income_2010'] = alter_data_2010.apply(lambda x: x['income_2010'] if x['hl_sta_2010'] == 1 else x['income_2010'] * 1.48, axis=1)
print(alter_data_2010)

          id   pn age_2010 gender_2010 hl_sta_2010  income_2010
0     000003  010       74        Male           2      3976.76
1     000003  020       72      Female           2      3976.76
2     010004  010       70        Male           2      7696.00
3     010004  040       64      Female           1      5200.00
4     010038  010       74        Male           1     21829.00
...      ...  ...      ...         ...         ...          ...
2071  917631  020       54      Female           2     23680.00
2072  918062  010       61      Female           2     60927.16
2073  918944  010       52        Male           2    171680.00
2074  920363  020       70      Female           1     30000.00
2075  920639  010       51        Male           3         0.00

[2076 rows x 6 columns]


In [14]:
# 2012, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2012\h12da\H12A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2012\h12da\H12C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2012\h12da\H12PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2012\h12da\H12N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2012', 'gender_2012', 'hl_sta_2012', 'mdc_expen_2012']
data_2012_R = pd.DataFrame(columns=cols)
for i in range(20554):
    age = int_limit(''.join(A_R[(66 * i + 42) : (66 * i + 45)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(759 * i + 33) : (759 * i + 36)]), 300)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(''.join(N_R[759 * i + 49]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(759 * i + 329) : (759 * i + 335)]), 400000)
    LTC_freq = { '1': 12, '2': 4, '3': 52, '4': 1, '6': 1 }.get(''.join(N_R[759 * i + 348]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(66 * i) : (66 * i + 6)]),
        'pn': ''.join(A_R[(66 * i + 6) : (66 * i + 9)]),
        'age_2012': age,
        'gender_2012': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[471 * i + 20]), None),
        'hl_sta_2012': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[360 * i + 23]), None),
        'mdc_expen_2012': mdc_expen
    }, index=[0])
    data_2012_R = pd.concat([data_2012_R, new], ignore_index=True)

alter_data_2012_R = data_2012_R.dropna()
maps = alter_data_2012_R['mdc_expen_2012'].groupby(alter_data_2012_R['id']).mean()
alter_data_2012_R = alter_data_2012_R.drop(['mdc_expen_2012'], axis=1)
alter_data_2012_R.merge(maps, how='inner', on='id')

# 2012, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2012\h12da\H12Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2012']
data_2012_H = pd.DataFrame(columns=cols)
for i in range(14316):
    ss = int_limit(''.join(Q_H[(2611 * i + 368) : (2611 * i + 372)]), 9998)
    p = int_limit(''.join(Q_H[(2611 * i + 949) : (2611 * i + 956)]), 9999998)
    o = int_limit(''.join(Q_H[(2611 * i + 1070) : (2611 * i + 1077)]), 9999998)
    ann = int_limit(''.join(Q_H[(2611 * i + 1263) : (2611 * i + 1270)]), 9999998)
    un_c = int_limit(''.join(Q_H[(2611 * i + 278) : (2611 * i + 286)]), 99999998)
    w_c = int_limit(''.join(Q_H[(2611 * i + 324) : (2611 * i + 331)]), 9999998)
    ssi = int_limit(''.join(Q_H[(2611 * i + 419) : (2611 * i + 423)]), 9998)
    wel = int_limit(''.join(Q_H[(2611 * i + 444) : (2611 * i + 450)]), 999998)
    vbmp = int_limit(''.join(Q_H[(2611 * i + 466) : (2611 * i + 470)]), 9998)
    li1 = int_limit(''.join(Q_H[(2611 * i + 31) : (2611 * i + 39)]), 99999998)
    li2 = int_limit(''.join(Q_H[(2611 * i + 56) : (2611 * i + 64)]), 99999998)
    li3 = int_limit(''.join(Q_H[(2611 * i + 81) : (2611 * i + 89)]), 99999998)
    li4 = int_limit(''.join(Q_H[(2611 * i + 106) : (2611 * i + 114)]), 99999998)
    li5 = int_limit(''.join(Q_H[(2611 * i + 130) : (2611 * i + 138)]), 99999998)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(2611 * i) : (2611 * i + 6)]),
        'subhh': ''.join(Q_H[2611 * i + 6]),
        'income_2012': income
    }, index=[0])
    data_2012_H = pd.concat([data_2012_H, new], ignore_index=True)
    
alter_data_2012_H = data_2012_H.dropna()
maps = alter_data_2012_H['income_2012'].groupby(alter_data_2012_H['id']).mean()

alter_data_2012 = alter_data_2012_R.merge(maps, how='inner', on='id')
alter_data_2012['income_2012'] = alter_data_2012.apply(lambda x: x['income_2012'] if x['hl_sta_2012'] == 1 else x['income_2012'] * 1.48, axis=1)
print(alter_data_2012)

          id   pn age_2012 gender_2012 hl_sta_2012  income_2012
0     000003  010       76        Male           2      1924.00
1     010004  040       66      Female           1         0.00
2     010038  010       75        Male           1      1900.00
3     010050  010       71      Female           2    124799.52
4     010097  040       64      Female           2      2220.00
...      ...  ...      ...         ...         ...          ...
1340  916841  010       56      Female           3     41440.00
1341  916871  020       67        Male           2    118400.00
1342  917056  020       49      Female           2     87320.00
1343  919069  010       60      Female           1         0.00
1344  919924  020       49      Female           2    162800.00

[1345 rows x 6 columns]


In [19]:
# 2014, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2014\h14da\H14A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2014\h14da\H14C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2014\h14da\H14PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2014\h14da\H14N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2014', 'gender_2014', 'hl_sta_2014', 'mdc_expen_2014']
data_2014_R = pd.DataFrame(columns=cols)
for i in range(18747):
    age = int_limit(''.join(A_R[(67 * i + 43) : (67 * i + 46)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(737 * i + 33) : (737 * i + 37)]), 6000)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(''.join(N_R[644 * i + 37]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(737 * i + 313) : (737 * i + 319)]), 100000)
    LTC_freq = { '1': 12, '2': 4, '3': 52, '4': 1, '6': 1 }.get(''.join(N_R[737 * i + 319]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(67 * i) : (67 * i + 6)]),
        'pn': ''.join(A_R[(67 * i + 6) : (67 * i + 9)]),
        'age_2014': age,
        'gender_2014': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[474 * i + 20]), None),
        'hl_sta_2014': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[317 * i + 23]), None),
        'mdc_expen_2014': mdc_expen
    }, index=[0])
    data_2014_R = pd.concat([data_2014_R, new], ignore_index=True)

alter_data_2014_R = data_2014_R.dropna()
maps = alter_data_2014_R['mdc_expen_2014'].groupby(alter_data_2014_R['id']).mean()
alter_data_2014_R = alter_data_2014_R.drop(['mdc_expen_2014'], axis=1)
alter_data_2014_R.merge(maps, how='inner', on='id')

# 2014, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2014\h14da\H14Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2014']
data_2014_H = pd.DataFrame(columns=cols)
for i in range(13241):
    ss = int_limit(''.join(Q_H[(2454 * i + 348) : (2454 * i + 352)]), 9998)
    p = int_limit(''.join(Q_H[(2454 * i + 880) : (2454 * i + 887)]), 9999998)
    o = int_limit(''.join(Q_H[(2454 * i + 997) : (2454 * i + 1001)]), 9998)
    ann = int_limit(''.join(Q_H[(2454 * i + 1175) : (2454 * i + 1180)]), 99998)
    un_c = int_limit(''.join(Q_H[(2454 * i + 266) : (2454 * i + 271)]), 99998)
    w_c = int_limit(''.join(Q_H[(2454 * i + 306) : (2454 * i + 311)]), 99998)
    ssi = int_limit(''.join(Q_H[(2454 * i + 399) : (2454 * i + 403)]), 9998)
    wel = int_limit(''.join(Q_H[(2454 * i + 424) : (2454 * i + 430)]), 999998)
    vbmp = int_limit(''.join(Q_H[(2454 * i + 446) : (2454 * i + 450)]), 9998)
    li1 = int_limit(''.join(Q_H[(2454 * i + 31) : (2454 * i + 39)]), 99999998)
    li2 = int_limit(''.join(Q_H[(2454 * i + 56) : (2454 * i + 64)]), 99999998)
    li3 = int_limit(''.join(Q_H[(2454 * i + 81) : (2454 * i + 89)]), 99999998)
    li4 = int_limit(''.join(Q_H[(2454 * i + 106) : (2454 * i + 112)]), 999998)
    li5 = int_limit(''.join(Q_H[(2454 * i + 128) : (2454 * i + 134)]), 999998)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(2454 * i) : (2454 * i + 6)]),
        'subhh': ''.join(Q_H[2454 * i + 6]),
        'income_2014': income
    }, index=[0])
    data_2014_H = pd.concat([data_2014_H, new], ignore_index=True)
    
alter_data_2014_H = data_2014_H.dropna()
maps = alter_data_2014_H['income_2014'].groupby(alter_data_2014_H['id']).mean()

alter_data_2014 = alter_data_2014_R.merge(maps, how='inner', on='id')
alter_data_2014['income_2014'] = alter_data_2014.apply(lambda x: x['income_2014'] if x['hl_sta_2014'] == 1 else x['income_2014'] * 1.48, axis=1)
print(alter_data_2014)

          id   pn age_2014 gender_2014 hl_sta_2014  income_2014
0     000003  020       75      Female           2      4292.00
1     010004  040       68      Female           1      1600.00
2     010038  010       77        Male           2      2812.00
3     010038  040       71      Female           1      1900.00
4     010097  040       67      Female           1      1400.00
...      ...  ...      ...         ...         ...          ...
2710  920078  010       56        Male           1     48250.00
2711  920331  010       57        Male           3      3076.92
2712  920363  020       74      Female           1     30000.00
2713  920538  010       65        Male           1      1641.00
2714  920760  020       74        Male           1         0.00

[2715 rows x 6 columns]


In [16]:
# 2016, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2016\h16da\H16A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2016\h16da\H16C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2016\h16da\H16PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2016\h16da\H16N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2016', 'gender_2016', 'hl_sta_2016', 'mdc_expen_2016']
data_2016_R = pd.DataFrame(columns=cols)
for i in range(20912):
    age = int_limit(''.join(A_R[(70 * i + 43) : (70 * i + 46)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(785 * i + 35) : (785 * i + 39)]), 3500)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(''.join(N_R[785 * i + 39]), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(785 * i + 338) : (785 * i + 344)]), 120000)
    LTC_freq = { '1': 12, '2': 4, '3': 52, '4': 1, '6': 1 }.get(''.join(N_R[785 * i + 344]), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(70 * i) : (70 * i + 6)]),
        'pn': ''.join(A_R[(70 * i + 6) : (70 * i + 9)]),
        'age_2016': age,
        'gender_2016': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[478 * i + 20]), None),
        'hl_sta_2016': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[341 * i + 22]), None),
        'mdc_expen_2016': mdc_expen
    }, index=[0])
    data_2016_R = pd.concat([data_2016_R, new], ignore_index=True)

alter_data_2016_R = data_2016_R.dropna()
maps = alter_data_2016_R['mdc_expen_2016'].groupby(alter_data_2016_R['id']).mean()
alter_data_2016_R = alter_data_2016_R.drop(['mdc_expen_2016'], axis=1)
alter_data_2016_R.merge(maps, how='inner', on='id')

# 2016, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2016\h16da\H16Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2016']
data_2016_H = pd.DataFrame(columns=cols)
for i in range(15023):
    ss = int_limit(''.join(Q_H[(2367 * i + 344) : (2367 * i + 348)]), 9998)
    p = int_limit(''.join(Q_H[(2367 * i + 903) : (2367 * i + 908)]), 99998)
    o = int_limit(''.join(Q_H[(2367 * i + 1015) : (2367 * i + 1019)]), 9998)
    ann = int_limit(''.join(Q_H[(2367 * i + 1188) : (2367 * i + 1193)]), 99998)
    un_c = int_limit(''.join(Q_H[(2367 * i + 264) : (2367 * i + 269)]), 99998)
    w_c = int_limit(''.join(Q_H[(2367 * i + 304) : (2367 * i + 309)]), 99998)
    ssi = int_limit(''.join(Q_H[(2367 * i + 395) : (2367 * i + 399)]), 9998)
    wel = int_limit(''.join(Q_H[(2367 * i + 420) : (2367 * i + 425)]), 99998)
    vbmp = int_limit(''.join(Q_H[(2367 * i + 441) : (2367 * i + 446)]), 99998)
    li1 = int_limit(''.join(Q_H[(2367 * i + 31) : (2367 * i + 39)]), 99999998)
    li2 = int_limit(''.join(Q_H[(2367 * i + 56) : (2367 * i + 64)]), 99999998)
    li3 = int_limit(''.join(Q_H[(2367 * i + 81) : (2367 * i + 88)]), 9999998)
    li4 = int_limit(''.join(Q_H[(2367 * i + 105) : (2367 * i + 111)]), 999998)
    li5 = int_limit(''.join(Q_H[(2367 * i + 127) : (2367 * i + 133)]), 999998)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(2367 * i) : (2367 * i + 6)]),
        'subhh': ''.join(Q_H[2367 * i + 6]),
        'income_2016': income
    }, index=[0])
    data_2016_H = pd.concat([data_2016_H, new], ignore_index=True)
    
alter_data_2016_H = data_2016_H.dropna()
maps = alter_data_2016_H['income_2016'].groupby(alter_data_2016_H['id']).mean()

alter_data_2016 = alter_data_2016_R.merge(maps, how='inner', on='id')
alter_data_2016['income_2016'] = alter_data_2016.apply(lambda x: x['income_2016'] if x['hl_sta_2016'] == 1 else x['income_2016'] * 1.48, axis=1)
print(alter_data_2016)

          id   pn age_2016 gender_2016 hl_sta_2016  income_2016
0     010004  040       70      Female           1      1800.00
1     010038  010       79        Male           1      2000.00
2     010038  040       73      Female           1      2000.00
3     010050  010       75      Female           2    125025.96
4     010097  040       68      Female           1      1400.00
...      ...  ...      ...         ...         ...          ...
3005  920192  010       65      Female           2       828.80
3006  920335  010       66      Female           1    116398.00
3007  920760  020       76        Male           1         0.00
3008  920772  010       61        Male           2     74000.00
3009  959738  010       65        Male           2      2442.00

[3010 rows x 6 columns]


In [21]:
# 2018, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2018\h18da\H18A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2018\h18da\H18C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2018\h18da\H18PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2018\h18da\H18N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2018', 'gender_2018', 'hl_sta_2018', 'mdc_expen_2018']
data_2018_R = pd.DataFrame(columns=cols)
for i in range(17146):
    age = int_limit(''.join(A_R[(75 * i + 44) : (75 * i + 47)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(921 * i + 45) : (921 * i + 50)]), 12000, 0)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(str(int_limit(''.join(N_R[(921 * i + 50) : (921 * i + 52)]))), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(921 * i + 400) : (921 * i + 406)]), 100000, 0)
    LTC_freq = { '1': 12, '2': 4, '3': 52, '4': 1, '6': 1 }.get(str(int_limit(''.join(N_R[(921 * i + 406) : (921 * i + 408)]))), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(75 * i) : (75 * i + 6)]),
        'pn': ''.join(A_R[(75 * i + 6) : (75 * i + 9)]),
        'age_2018': age,
        'gender_2018': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[489 * i + 20]), None),
        'hl_sta_2018': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(str(int(''.join(C_R[(514 * i + 23) : (514 * i + 25)]))), None),
        'mdc_expen_2018': mdc_expen
    }, index=[0])
    data_2018_R = pd.concat([data_2018_R, new], ignore_index=True)

alter_data_2018_R = data_2018_R.dropna()
maps = alter_data_2018_R['mdc_expen_2018'].groupby(alter_data_2018_R['id']).mean()
alter_data_2018_R = alter_data_2018_R.drop(['mdc_expen_2018'], axis=1)
alter_data_2018_R.merge(maps, how='inner', on='id')

# 2018, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2018\h18da\H18Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2018']
data_2018_H = pd.DataFrame(columns=cols)
for i in range(12467):
    ss = int_limit(''.join(Q_H[(2602 * i + 358) : (2602 * i + 363)]), 99998)
    p = int_limit(''.join(Q_H[(2602 * i + 990) : (2602 * i + 995)]), 99998)
    o = int_limit(''.join(Q_H[(2602 * i + 1113) : (2602 * i + 1119)]), 999998)
    ann = int_limit(''.join(Q_H[(2602 * i + 1306) : (2602 * i + 1311)]), 99998)
    un_c = int_limit(''.join(Q_H[(2602 * i + 276) : (2602 * i + 281)]), 99998)
    w_c = int_limit(''.join(Q_H[(2602 * i + 318) : (2602 * i + 323)]), 99998)
    ssi = int_limit(''.join(Q_H[(2602 * i + 414) : (2602 * i + 419)]), 99998)
    wel = int_limit(''.join(Q_H[(2602 * i + 442) : (2602 * i + 447)]), 99998)
    vbmp = int_limit(''.join(Q_H[(2602 * i + 465) : (2602 * i + 469)]), 9998)
    li1 = int_limit(''.join(Q_H[(2602 * i + 36) : (2602 * i + 43)]), 9999998)
    li2 = int_limit(''.join(Q_H[(2602 * i + 61) : (2602 * i + 67)]), 999998)
    li3 = int_limit(''.join(Q_H[(2602 * i + 85) : (2602 * i + 92)]), 9999998)
    li4 = int_limit(''.join(Q_H[(2602 * i + 110) : (2602 * i + 116)]), 999998)
    li5 = int_limit(''.join(Q_H[(2602 * i + 133) : (2602 * i + 138)]), 99998)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(2602 * i) : (2602 * i + 6)]),
        'subhh': ''.join(Q_H[2602 * i + 6]),
        'income_2018': income
    }, index=[0])
    data_2018_H = pd.concat([data_2018_H, new], ignore_index=True)
    
alter_data_2018_H = data_2018_H.dropna()
maps = alter_data_2018_H['income_2018'].groupby(alter_data_2018_H['id']).mean()

alter_data_2018 = alter_data_2018_R.merge(maps, how='inner', on='id')
alter_data_2018['income_2018'] = alter_data_2018.apply(lambda x: x['income_2018'] if x['hl_sta_2018'] == 1 else x['income_2018'] * 1.48, axis=1)
print(alter_data_2018)

          id   pn age_2018 gender_2018 hl_sta_2018  income_2018
0     010004  040       72      Female           1      1500.00
1     010038  010       81        Male           1      2000.00
2     010038  040       75      Female           1      2000.00
3     010106  010       87        Male           2      5328.00
4     010106  020       85      Female           3      5328.00
...      ...  ...      ...         ...         ...          ...
2611  920335  010       68      Female           2     39227.40
2612  920353  010       69      Female           2      5884.48
2613  920690  010       63      Female           2      2296.96
2614  920697  010       68      Female           2         0.00
2615  923525  010       61      Female           2     55204.00

[2616 rows x 6 columns]


In [22]:
# 2020, 以 respondent 為單位的回答
A_R = np.array(list(open('D:\Desktop\imf\專題\data\\2020\h20da\H20A_R.DA', mode='r').read()))
C_R = np.array(list(open('D:\Desktop\imf\專題\data\\2020\h20da\H20C_R.DA', mode='r').read()))
PR_R = np.array(list(open('D:\Desktop\imf\專題\data\\2020\h20da\H20PR_R.DA', mode='r').read()))
N_R = np.array(list(open('D:\Desktop\imf\專題\data\\2020\h20da\H20N_R.DA', mode='r').read()))

cols = ['id', 'pn', 'age_2020', 'gender_2020', 'hl_sta_2020', 'mdc_expen_2020']
data_2020_R = pd.DataFrame(columns=cols)
for i in range(15723):
    age = int_limit(''.join(A_R[(75 * i + 44) : (75 * i + 47)]), 200)
    healthcare_AMT = float_limit(''.join(N_R[(796 * i + 43) : (796 * i + 48)]), 35170, 0)
    healthcare_freq = { '1': 12, '2': 4, '3': 2, '4': 1 }.get(str(int_limit(''.join(N_R[(796 * i + 48) : (796 * i + 50)]))), 0)
    healthcare = healthcare_AMT * healthcare_freq if healthcare_AMT else 0
    LTC_AMT = float_limit(''.join(N_R[(796 * i + 339) : (796 * i + 345)]), 100000, 0)
    LTC_freq = { '1': 12, '2': 4, '3': 52, '4': 1, '6': 1 }.get(str(int_limit(''.join(N_R[(796 * i + 345) : (796 * i + 347)]))), 0)
    LTC = LTC_AMT * LTC_freq if LTC_AMT else 0
    mdc_expen = healthcare + LTC if healthcare_AMT or LTC_AMT else None
    
    new = pd.DataFrame({
        'id': ''.join(A_R[(75 * i) : (75 * i + 6)]),
        'pn': ''.join(A_R[(75 * i + 6) : (75 * i + 9)]),
        'age_2020': age,
        'gender_2020': { '1': 'Male', '2': 'Female' }.get(''.join(PR_R[492 * i + 20]), None),
        'hl_sta_2020': { '1': 1, '2': 1, '3': 2, '4': 2, '5': 3 }.get(''.join(C_R[479 * i + 23]), None),
        'mdc_expen_2020': mdc_expen
    }, index=[0])
    data_2020_R = pd.concat([data_2020_R, new], ignore_index=True)

alter_data_2020_R = data_2020_R.dropna()
maps = alter_data_2020_R['mdc_expen_2020'].groupby(alter_data_2020_R['id']).mean()
alter_data_2020_R = alter_data_2020_R.drop(['mdc_expen_2020'], axis=1)
alter_data_2020_R.merge(maps, how='inner', on='id')

# 2020, 以 household 為單位的回答
Q_H = np.array(list(open('D:\Desktop\imf\專題\data\\2020\h20da\H20Q_H.DA', mode='r').read()))

cols = ['id', 'subhh', 'income_2020']
data_2020_H = pd.DataFrame(columns=cols)
for i in range(11490):
    ss = int_limit(''.join(Q_H[(2598 * i + 357) : (2598 * i + 362)]), 99998, 0)
    p = int_limit(''.join(Q_H[(2598 * i + 992) : (2598 * i + 997)]), 99998, 0)
    o = int_limit(''.join(Q_H[(2598 * i + 1115) : (2598 * i + 1121)]), 9998, 0)
    ann = int_limit(''.join(Q_H[(2598 * i + 1302) : (2598 * i + 1307)]), 99998, 0)
    un_c = int_limit(''.join(Q_H[(2598 * i + 277) : (2598 * i + 282)]), 99998, 0)
    w_c = int_limit(''.join(Q_H[(2598 * i + 318) : (2598 * i + 323)]), 99998, 0)
    ssi = int_limit(''.join(Q_H[(2598 * i + 413) : (2598 * i + 418)]), 9998, 0)
    wel = int_limit(''.join(Q_H[(2598 * i + 441) : (2598 * i + 446)]), 99998, 0)
    vbmp = int_limit(''.join(Q_H[(2598 * i + 463) : (2598 * i + 468)]), 99998, 0)
    li1 = int_limit(''.join(Q_H[(2598 * i + 35) : (2598 * i + 42)]), 9999998, 0)
    li2 = int_limit(''.join(Q_H[(2598 * i + 60) : (2598 * i + 67)]), 999998, 0)
    li3 = int_limit(''.join(Q_H[(2598 * i + 85) : (2598 * i + 92)]), 999998, 0)
    li4 = int_limit(''.join(Q_H[(2598 * i + 110) : (2598 * i + 116)]), 999998, 0)
    li5 = int_limit(''.join(Q_H[(2598 * i + 133) : (2598 * i + 139)]), 999998, 0)
    filter_list = filter(None, [ss, p, o, ann, un_c, w_c, ssi, wel, vbmp, li1, li2, li3, li4, li5])
    income = sum(filter_list) if filter_list else None
    new = pd.DataFrame({
        'id': ''.join(Q_H[(2598 * i) : (2598 * i + 6)]),
        'subhh': ''.join(Q_H[2598 * i + 6]),
        'income_2020': income
    }, index=[0])
    data_2020_H = pd.concat([data_2020_H, new], ignore_index=True)
    
alter_data_2020_H = data_2020_H.dropna()
maps = alter_data_2020_H['income_2020'].groupby(alter_data_2020_H['id']).mean()

alter_data_2020 = alter_data_2020_R.merge(maps, how='inner', on='id')
alter_data_2020['income_2020'] = alter_data_2020.apply(lambda x: x['income_2020'] if x['hl_sta_2020'] == 1 else x['income_2020'] * 1.48, axis=1)
print(alter_data_2020)

          id   pn age_2020 gender_2020 hl_sta_2020  income_2020
0     010004  040       73      Female           2       2664.0
1     010013  040       72      Female           2      10656.0
2     010038  010       83        Male           1       2226.0
3     010038  040       77      Female           1       2226.0
4     010050  010       79      Female           2     143434.2
...      ...  ...      ...         ...         ...          ...
2425  920162  010       67      Female           1     281300.0
2426  920192  010       69      Female           2       1790.8
2427  920275  020       73        Male           2       1776.0
2428  920690  010       65      Female           3        740.0
2429  920697  010       70      Female           2          0.0

[2430 rows x 6 columns]
